In [1]:
import time, cv2, sys, os, torch
from threading import Thread
from djitellopy import Tello
import openai

# load environment variables
from dotenv import load_dotenv
load_dotenv()

openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
from transformers import ViltProcessor, ViltForQuestionAnswering

processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa")

In [ ]:
drone = Tello()

In [ ]:
objective = "Find the person who is wearing a blue cap"

In [ ]:
drone.streamon()
drone.takeoff()

In [ ]:
def query_the_environment(drone, question):
    image = drone.get_frame_read().frame
    encoding = processor(image, question, return_tensors="pt")
    outputs = model(**encoding)
    logits = outputs.logits
    idx = torch.sigmoid(logits).argmax(-1).item()
    return str(model.config.id2label[idx])


In [ ]:
def explore(drone):
    pass

In [ ]:
# Generate next action prompt
def next_action_prompt(prompt_template, objective, environment_context, previous_command=None):
    prompt = prompt_template.format(objective=objective, environment_context=environment_context, previous_command=previous_command)
    return prompt

In [ ]:
def vqa_prompt(prompt_template, objective):
    prompt = prompt_template.format(objective=objective)
    return prompt

In [ ]:
def prompt(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        temperature=0.5,
        max_tokens=300,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=10,
        stop=["\n"]
    )
    return response.choices[0].text